#TopEditor - traitements NER et enrichissements des textes TEI

Le dossier du projet se trouve ici :
```
/content/drive/MyDrive/Colab Notebooks/TopEditor/
```
Description des sous-dossiers associés aux traitements:
* **NERmodels** : les modèles NER produits de reperage d'entités nommées (et autres)

* **data/TEI/Final/input**: fichiers TEI stylé via Word customisé Metopes et converti en schéma métopes (via XMLMind)

* **data/TEI/Final/clean**: fichiers TEI du dossier input legerement "nettoyés" manuellement du bruit produit par Word après consignes Métopes

* **data/TEI/Final/tagged**: fichiers TEI avec balisage automatisé via un modèle NER custom

* **data/TEI/Final/reference_pers**: fichiers tableurs avec les identifiants personnes à désambiguiser manuellement

* **data/TEI/Final/reference_place**: fichiers tableurs avec les identifiants place (sens restreint) à désambiguiser manuellement

* **data/TEI/Final/tagged/updated_person**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes

* **data/TEI/Final/tagged/updated_person_upated_place**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes et pour les lieux

* **data/TEI/Final/tagged_corrected_Sara**: fichiers TEI dans updated_person_upated_place avec la correction manuelle par Sara via Oxygène des balises fautives

* **data/TEI/Final/tagged_corrected_Sara/TagAugmented** fichiers dans tagged_corrected_Sara avec un balisage automatisé par reglès des dates déjà reperées par le modèle NER et pas du tout repérées




## Limpieza manual de documentos TEI en entrada (input -> clean)

Nettoyage de bruit généré par MS Word, fait par Regexp via notepad++:

- ID « CellXY » has already been defined > supprimer les éléments <rendition> redondant : il s’agit du bruit fait par Word pour les déclarations CSS sur les rendus des cellules de tableaux
```
<rendition.*?>.*?</rendition>
```
ou
```
<rendition.*?>.*?</rendition.*?>
```

selectionner mode regexp et cocher matches newline ; effacer le restant manuellement

- bad value for attribute ID > supprimer les éléments <anchor>
```
<anchor.*?/>
```
ou
```
<anchor.*?></anchor.*?>
```

selectionner mode regexp et cocher matches newline

A ce stade, il est possible de compléter les trois métadonnées manquantes d'après le modèle Métopes:

```
<text type="##" xml:id="text"> ## par "chapter"
<language ident="##-##"/> ##-## par "es-ES" (plutôt "fr-FR" ?)

```

Ensuite, il faut ouvrir les fichiers nettoyer dans XMLMind (Pluco, environment TopEditor) et vérifier et corriger manuellement les erreurs restantes (par exemple, doublons dans les identifiants des notes de bas de page ou des metadonnées manquantes comme langue ou type du document).






##Anotaciones NER (clean -> tagged)

Sobre los parrafos en p con dos valores posibles de rend, TEI_localparaDonnees y TEI_localparaData, luego de text, esto excluye las figuras y sus leyendas asi que los cuadros de fuentes primarias (en espagnol antiguo).

###Instalaciones

In [1]:
!pip install -U pip setuptools wheel

In [2]:
!pip install -U 'spacy[cuda12x,transformers,lookups]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 133.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 153.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!python -m spacy download fr_dep_news_trf
#!python -m spacy download fr_core_news_sm
#!python -m spacy download es_dep_news_trf
#!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.7/397.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('fr_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies

In [4]:
!python -m spacy info

/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.8.4                         
Location         /usr/local/lib/python3.11/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.11.11                       
Pipelines        fr_dep_news_trf (3.8.0), en_core_web_sm (3.7.1)



### Aplicar un modelo NER a cada parrafo en los documentos TEI y a partir de correspondencias, traducir las categorias NER en forma de tags XML en archivo de salida

Informacion sobre las correspondencias: https://docs.google.com/document/d/1LHSwAEsh6_bNxRiUdkgvjYaPqbO-zjdvqNTrh2PaOxo/edit#heading=h.jo4bp94cw2ay

Ce traitement est équivalent à celui fourni par **Oryx**, néanmoins après comparaison entre les sorties, on perd plus ou moins la meme quantité d'information (balises notes) avec l'un ou l'autre.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import spacy
from lxml import etree

# Charger le modèle spaCy
model_spacy_gpu_trf = "/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/model-best"
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/TopEditor"
nlp = spacy.load(gdrive_path + model_spacy_gpu_trf)

# Dossier contenant les fichiers texte
dossier_textes = gdrive_path + '/data/TEI/'

# Dictionnaire pour mapper les catégories NER aux balises TEI
ner_to_tei = {
    "PER": ("persName", {"ref": "x"}),
    "ORG": ("orgName", {}),
    "Fecha_periodos_eventos": ("date", {"to":"x","from":"x"}),
    "LOC": ("placeName", {}),
    "rol_pers_familia": ("rs", {"type":"family_role"}),
    "tipo_edificio": ("rs", {"type":"building_type"}),
    "nombre_calle": ("rs", {"type":"street_name"}),
    "nombre_collacion": ("rs", {"type":"area_name"}),
    "nombre_CTMB": ("rs", {"type":"poi_name"}),
    "modo_pago": ("rs", {"type":"payment_mode"}),
    "tipo_casa": ("rs", {"type":"house_type"}),
    "Alias_persona": ("name", {}),
    "piezas_casas": ("rs", {"type":"house_part_type"}),
    "TipoContratoLT": ("rs", {"type":"longterm_contract_type"}),
    "medidas":("rs", {"type":"measurements"}),
    "Oficio":  ("rs", {"type":"profession"}),
    "Titulo_role_statut":  ("roleName", {})
}

from lxml import etree

# Fonction pour baliser les entités nommées dans le texte sous forme de sous-éléments XML
def baliser_entites_xml(element, texte, ents, ner_to_tei):
    dernier_indice = 0
    # Créer une variable pour contenir le nouveau texte
    nouveau_contenu = []

    # Créer un identifiant temporaire pour chaque segment
    segment_id = 1

    # Parcourir chaque entité nommée détectée
    for ent in ents:
     if(ent.label_ in ner_to_tei):
        # Ajouter le texte avant l'entité, avec un identifiant temporaire
        if dernier_indice < ent.start_char:
            text_before = texte[dernier_indice:ent.start_char]
            if text_before.strip():  # Ne pas ajouter de texte vide
                nouveau_contenu.append((f"txt_{segment_id}", text_before))
                segment_id += 1

        # Récupérer le tag et les attributs à partir du dictionnaire
        tag, attributes = ner_to_tei.get(ent.label_, ("ent", {"type": ent.label_}))  # Par défaut <ent>

        # Créer l'élément XML pour l'entité
        ent_element = etree.Element(tag, attributes)
        ent_element.text = ent.text
        nouveau_contenu.append((f"ent_{segment_id}", ent_element))
        segment_id += 1

        # Mettre à jour l'indice
        dernier_indice = ent.end_char

    # Ajouter le texte restant après la dernière entité
    if dernier_indice < len(texte):
        text_after = texte[dernier_indice:]
        if text_after.strip():
            nouveau_contenu.append((f"txt_{segment_id}", text_after))

    # Effacer le contenu de l'élément tout en conservant ses attributs
    element.clear()

    # Ajouter les éléments (textes et entités) dans l'ordre dans l'élément <p>
    is_first = True
    for _, partie in nouveau_contenu:
        if isinstance(partie, str):
            if is_first:
                element.text = partie  # Premier texte avant les entités
                is_first = False
            else:
                # Utiliser `tail` pour ajouter le texte après un élément
                if len(element) > 0:
                    element[-1].tail = (element[-1].tail or '') + partie
                else:
                    element.text = (element.text or '') + partie
        else:
            # Ajouter l'élément <ent> à l'élément <p>
            element.append(partie)

# Fonction pour obtenir le texte complet d'une balise XML
def get_full_text(element):
    return ''.join(element.itertext())

# Lire et traiter les fichiers XML
for fichier in os.listdir(dossier_textes+"Final/clean/"):
    if fichier.endswith('.xml'):
        chemin_fichier = os.path.join(dossier_textes+"Final/clean/", fichier)

        # Charger et parser le fichier XML avec lxml
        tree = etree.parse(chemin_fichier)

        root = tree.getroot()

        # Namespace TEI
        namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

        # Trouver le body
        body = root.find('.//tei:body', namespaces=namespace)

        # Extraire les textes dans les balises <p> du body avec leurs attributs xml:id
        #paragraphs = body.findall('.//tei:p', namespaces=namespace)
        paragraphs = body.findall(".//tei:p[@rend='TEI_localparaDonnees']", namespaces=namespace) + body.findall(".//tei:p[@rend='TEI_localparaDate']", namespaces=namespace)

        # Parcourir chaque paragraphe et baliser les entités nommées
        for p in paragraphs:
            # Récupérer le texte du paragraphe
            text = get_full_text(p)

            # Conserver les attributs de la balise <p>
            attributs_p = dict(p.attrib)  # Correctement copier les attributs

            # Baliser les entités nommées en passant les entités reconnues par spaCy et le dictionnaire
            baliser_entites_xml(p, text, nlp(text).ents, ner_to_tei)

            # Restaurer les attributs après modification
            p.attrib.update(attributs_p)

        # Sauvegarder le fichier XML modifié avec les balises d'entités
        sortie_fichier = os.path.join(dossier_textes+"Final/tagged/", 'semtags_' + fichier)

        # Sauvegarder avec `lxml` et conserver les namespaces
        tree.write(sortie_fichier, encoding="UTF-8", xml_declaration=True, pretty_print=True)

        print(f"Fichier {fichier} traité et sauvegardé sous {sortie_fichier}.")


/usr/local/lib/python3.11/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fileli

Fichier 2combiné_T2 clj. San Pedro_clean.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml.


Ver notebook de "control".

**ATTENTION: il y a une perte systématique d'information des notes de bas de page.**